In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


In [3]:

# How you can start mapping words into vector spaces
# and how you can start looking at sentiment even visualizing how your model has lernt sentiment from the sense of words
import tensorflow as tf

print (tf.__version__)
tf.enable_eager_execution()  #if needed

!pip install -q tensorflow-datasets

import tensorflow_datasets as tfds

imdb, info = tfds.load ("imdb_reviews", with_info=True, as_supervised=True)


1.15.0


You should consider upgrading via the 'c:\users\leila\anaconda3\pythonw.exe -m pip install --upgrade pip' command.


In [4]:

import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in python3 instead of just s.numpy()
for s,l in train_data :
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data :
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [7]:

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

# To reverse indexs to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])



# Define a neural network
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),  #--> output_shape = vocab_size, embeddd_dim
    tf.keras.layers.Flatten(),     #OR     --> output_shape = 1920  which is equal to vocab_size* embeddd_dim
#     tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

model.compile( loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

# fitting
history = model.fit(padded, training_labels_final,
          epochs=10,
          validation_data=(testing_padded, testing_labels_final))

e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)



# to visualize the vector data
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index [word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


# To download the files
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

# To visualizing data go to http://projector.tensorflow.org/ load vecs.tsv for the step1 and meta.tsv for step 2.
#     tick Sphereize data  box to see the result

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'
Model: "sequenti

25000/25000 [==============================] - ETA: 24s - loss: 0.3430 - acc: 0.87 - ETA: 8s - loss: 0.2342 - acc: 0.9180 - ETA: 7s - loss: 0.2289 - acc: 0.921 - ETA: 7s - loss: 0.2264 - acc: 0.918 - ETA: 6s - loss: 0.2341 - acc: 0.916 - ETA: 7s - loss: 0.2355 - acc: 0.914 - ETA: 7s - loss: 0.2358 - acc: 0.915 - ETA: 7s - loss: 0.2353 - acc: 0.915 - ETA: 7s - loss: 0.2382 - acc: 0.914 - ETA: 7s - loss: 0.2385 - acc: 0.915 - ETA: 7s - loss: 0.2397 - acc: 0.915 - ETA: 7s - loss: 0.2385 - acc: 0.915 - ETA: 7s - loss: 0.2371 - acc: 0.915 - ETA: 7s - loss: 0.2398 - acc: 0.914 - ETA: 7s - loss: 0.2412 - acc: 0.913 - ETA: 6s - loss: 0.2441 - acc: 0.911 - ETA: 6s - loss: 0.2448 - acc: 0.911 - ETA: 6s - loss: 0.2429 - acc: 0.912 - ETA: 6s - loss: 0.2434 - acc: 0.910 - ETA: 6s - loss: 0.2464 - acc: 0.909 - ETA: 6s - loss: 0.2441 - acc: 0.910 - ETA: 6s - loss: 0.2459 - acc: 0.909 - ETA: 6s - loss: 0.2483 - acc: 0.908 - ETA: 6s - loss: 0.2475 - acc: 0.907 - ETA: 6s - loss: 0.2486 - acc: 0.906 - ET

25000/25000 [==============================] - ETA: 21s - loss: 0.1069 - acc: 0.96 - ETA: 10s - loss: 0.1398 - acc: 0.95 - ETA: 7s - loss: 0.1150 - acc: 0.9760 - ETA: 7s - loss: 0.1067 - acc: 0.975 - ETA: 6s - loss: 0.1081 - acc: 0.977 - ETA: 6s - loss: 0.1094 - acc: 0.975 - ETA: 6s - loss: 0.1061 - acc: 0.977 - ETA: 6s - loss: 0.1028 - acc: 0.978 - ETA: 6s - loss: 0.0995 - acc: 0.980 - ETA: 6s - loss: 0.1032 - acc: 0.979 - ETA: 6s - loss: 0.1012 - acc: 0.979 - ETA: 6s - loss: 0.1016 - acc: 0.979 - ETA: 6s - loss: 0.1000 - acc: 0.979 - ETA: 5s - loss: 0.0971 - acc: 0.981 - ETA: 5s - loss: 0.0982 - acc: 0.980 - ETA: 5s - loss: 0.0981 - acc: 0.980 - ETA: 5s - loss: 0.0968 - acc: 0.981 - ETA: 5s - loss: 0.0968 - acc: 0.981 - ETA: 5s - loss: 0.0973 - acc: 0.981 - ETA: 5s - loss: 0.0966 - acc: 0.981 - ETA: 5s - loss: 0.0957 - acc: 0.981 - ETA: 5s - loss: 0.0968 - acc: 0.980 - ETA: 5s - loss: 0.0963 - acc: 0.981 - ETA: 5s - loss: 0.0974 - acc: 0.980 - ETA: 5s - loss: 0.0971 - acc: 0.979 - ET

25000/25000 [==============================] - ETA: 12s - loss: 0.0387 - acc: 1.00 - ETA: 7s - loss: 0.0254 - acc: 1.0000 - ETA: 7s - loss: 0.0238 - acc: 0.997 - ETA: 6s - loss: 0.0241 - acc: 0.996 - ETA: 6s - loss: 0.0223 - acc: 0.997 - ETA: 6s - loss: 0.0223 - acc: 0.998 - ETA: 6s - loss: 0.0220 - acc: 0.998 - ETA: 6s - loss: 0.0219 - acc: 0.998 - ETA: 6s - loss: 0.0219 - acc: 0.998 - ETA: 6s - loss: 0.0212 - acc: 0.999 - ETA: 6s - loss: 0.0219 - acc: 0.998 - ETA: 5s - loss: 0.0220 - acc: 0.998 - ETA: 5s - loss: 0.0217 - acc: 0.998 - ETA: 5s - loss: 0.0225 - acc: 0.998 - ETA: 5s - loss: 0.0237 - acc: 0.998 - ETA: 5s - loss: 0.0250 - acc: 0.997 - ETA: 5s - loss: 0.0245 - acc: 0.997 - ETA: 5s - loss: 0.0243 - acc: 0.997 - ETA: 5s - loss: 0.0243 - acc: 0.997 - ETA: 5s - loss: 0.0240 - acc: 0.997 - ETA: 5s - loss: 0.0237 - acc: 0.997 - ETA: 5s - loss: 0.0238 - acc: 0.997 - ETA: 5s - loss: 0.0242 - acc: 0.997 - ETA: 5s - loss: 0.0240 - acc: 0.997 - ETA: 5s - loss: 0.0236 - acc: 0.997 - ET

25000/25000 [==============================] - ETA: 17s - loss: 0.0066 - acc: 1.00 - ETA: 9s - loss: 0.0164 - acc: 0.9948 - ETA: 7s - loss: 0.0108 - acc: 0.997 - ETA: 6s - loss: 0.0093 - acc: 0.998 - ETA: 6s - loss: 0.0087 - acc: 0.998 - ETA: 6s - loss: 0.0082 - acc: 0.999 - ETA: 6s - loss: 0.0080 - acc: 0.999 - ETA: 6s - loss: 0.0078 - acc: 0.999 - ETA: 6s - loss: 0.0076 - acc: 0.999 - ETA: 6s - loss: 0.0075 - acc: 0.999 - ETA: 5s - loss: 0.0073 - acc: 0.999 - ETA: 5s - loss: 0.0072 - acc: 0.999 - ETA: 5s - loss: 0.0071 - acc: 0.999 - ETA: 5s - loss: 0.0069 - acc: 0.999 - ETA: 5s - loss: 0.0068 - acc: 0.999 - ETA: 5s - loss: 0.0066 - acc: 0.999 - ETA: 5s - loss: 0.0066 - acc: 0.999 - ETA: 5s - loss: 0.0065 - acc: 0.999 - ETA: 5s - loss: 0.0064 - acc: 0.999 - ETA: 5s - loss: 0.0064 - acc: 0.999 - ETA: 5s - loss: 0.0064 - acc: 0.999 - ETA: 5s - loss: 0.0063 - acc: 0.999 - ETA: 5s - loss: 0.0063 - acc: 0.999 - ETA: 5s - loss: 0.0063 - acc: 0.999 - ETA: 5s - loss: 0.0062 - acc: 0.999 - ET

25000/25000 [==============================] - ETA: 8s - loss: 0.0025 - acc: 1.000 - ETA: 6s - loss: 0.0022 - acc: 1.000 - ETA: 6s - loss: 0.0024 - acc: 1.000 - ETA: 6s - loss: 0.0024 - acc: 1.000 - ETA: 6s - loss: 0.0024 - acc: 1.000 - ETA: 6s - loss: 0.0024 - acc: 1.000 - ETA: 6s - loss: 0.0023 - acc: 1.000 - ETA: 6s - loss: 0.0023 - acc: 1.000 - ETA: 6s - loss: 0.0023 - acc: 1.000 - ETA: 6s - loss: 0.0023 - acc: 1.000 - ETA: 6s - loss: 0.0023 - acc: 1.000 - ETA: 6s - loss: 0.0022 - acc: 1.000 - ETA: 6s - loss: 0.0022 - acc: 1.000 - ETA: 6s - loss: 0.0022 - acc: 1.000 - ETA: 6s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0021 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA: 5s - loss: 0.0022 - acc: 1.000 - ETA

25000/25000 [==============================] - ETA: 10s - loss: 0.0013 - acc: 1.00 - ETA: 7s - loss: 0.0010 - acc: 1.0000 - ETA: 7s - loss: 0.0010 - acc: 1.000 - ETA: 7s - loss: 8.9029e-04 - acc: 1.000 - ETA: 7s - loss: 9.1328e-04 - acc: 1.000 - ETA: 6s - loss: 9.6026e-04 - acc: 1.000 - ETA: 6s - loss: 9.5210e-04 - acc: 1.000 - ETA: 6s - loss: 9.1713e-04 - acc: 1.000 - ETA: 6s - loss: 9.1788e-04 - acc: 1.000 - ETA: 6s - loss: 9.3112e-04 - acc: 1.000 - ETA: 6s - loss: 9.1869e-04 - acc: 1.000 - ETA: 6s - loss: 9.1779e-04 - acc: 1.000 - ETA: 6s - loss: 8.9777e-04 - acc: 1.000 - ETA: 6s - loss: 8.9608e-04 - acc: 1.000 - ETA: 6s - loss: 8.8530e-04 - acc: 1.000 - ETA: 6s - loss: 8.9872e-04 - acc: 1.000 - ETA: 6s - loss: 9.0054e-04 - acc: 1.000 - ETA: 6s - loss: 9.0214e-04 - acc: 1.000 - ETA: 6s - loss: 8.9810e-04 - acc: 1.000 - ETA: 5s - loss: 8.9521e-04 - acc: 1.000 - ETA: 5s - loss: 8.8623e-04 - acc: 1.000 - ETA: 5s - loss: 8.8280e-04 - acc: 1.000 - ETA: 5s - loss: 8.8795e-04 - acc: 1.000 

25000/25000 [==============================] - ETA: 11s - loss: 7.1353e-04 - acc: 1.00 - ETA: 7s - loss: 4.8482e-04 - acc: 1.0000 - ETA: 7s - loss: 4.8814e-04 - acc: 1.000 - ETA: 6s - loss: 4.9071e-04 - acc: 1.000 - ETA: 6s - loss: 5.5706e-04 - acc: 1.000 - ETA: 6s - loss: 5.6340e-04 - acc: 1.000 - ETA: 6s - loss: 5.4925e-04 - acc: 1.000 - ETA: 5s - loss: 5.4477e-04 - acc: 1.000 - ETA: 6s - loss: 5.4310e-04 - acc: 1.000 - ETA: 5s - loss: 5.2626e-04 - acc: 1.000 - ETA: 5s - loss: 5.2165e-04 - acc: 1.000 - ETA: 5s - loss: 5.2613e-04 - acc: 1.000 - ETA: 5s - loss: 5.2108e-04 - acc: 1.000 - ETA: 5s - loss: 5.3600e-04 - acc: 1.000 - ETA: 5s - loss: 5.3169e-04 - acc: 1.000 - ETA: 5s - loss: 5.2635e-04 - acc: 1.000 - ETA: 5s - loss: 5.2915e-04 - acc: 1.000 - ETA: 5s - loss: 5.2198e-04 - acc: 1.000 - ETA: 5s - loss: 5.2788e-04 - acc: 1.000 - ETA: 5s - loss: 5.3205e-04 - acc: 1.000 - ETA: 5s - loss: 5.3017e-04 - acc: 1.000 - ETA: 5s - loss: 5.2562e-04 - acc: 1.000 - ETA: 5s - loss: 5.2038e-04 -

25000/25000 [==============================] - ETA: 9s - loss: 5.3140e-04 - acc: 1.000 - ETA: 6s - loss: 3.6312e-04 - acc: 1.000 - ETA: 6s - loss: 3.1959e-04 - acc: 1.000 - ETA: 6s - loss: 3.1375e-04 - acc: 1.000 - ETA: 6s - loss: 2.9839e-04 - acc: 1.000 - ETA: 7s - loss: 2.9277e-04 - acc: 1.000 - ETA: 6s - loss: 2.9515e-04 - acc: 1.000 - ETA: 6s - loss: 2.9411e-04 - acc: 1.000 - ETA: 6s - loss: 3.0248e-04 - acc: 1.000 - ETA: 6s - loss: 3.0804e-04 - acc: 1.000 - ETA: 6s - loss: 3.0716e-04 - acc: 1.000 - ETA: 6s - loss: 2.9781e-04 - acc: 1.000 - ETA: 6s - loss: 3.0441e-04 - acc: 1.000 - ETA: 6s - loss: 3.0186e-04 - acc: 1.000 - ETA: 6s - loss: 3.3470e-04 - acc: 1.000 - ETA: 6s - loss: 3.3280e-04 - acc: 1.000 - ETA: 6s - loss: 3.2622e-04 - acc: 1.000 - ETA: 6s - loss: 3.2403e-04 - acc: 1.000 - ETA: 6s - loss: 3.2021e-04 - acc: 1.000 - ETA: 5s - loss: 3.1768e-04 - acc: 1.000 - ETA: 5s - loss: 3.1568e-04 - acc: 1.000 - ETA: 5s - loss: 3.0972e-04 - acc: 1.000 - ETA: 5s - loss: 3.1454e-04 - 

25000/25000 [==============================] - ETA: 10s - loss: 1.9851e-04 - acc: 1.00 - ETA: 5s - loss: 1.6828e-04 - acc: 1.0000 - ETA: 5s - loss: 1.4907e-04 - acc: 1.000 - ETA: 6s - loss: 1.5901e-04 - acc: 1.000 - ETA: 5s - loss: 1.6336e-04 - acc: 1.000 - ETA: 5s - loss: 1.6424e-04 - acc: 1.000 - ETA: 5s - loss: 1.6492e-04 - acc: 1.000 - ETA: 5s - loss: 1.6534e-04 - acc: 1.000 - ETA: 5s - loss: 1.6596e-04 - acc: 1.000 - ETA: 6s - loss: 1.7118e-04 - acc: 1.000 - ETA: 6s - loss: 1.7007e-04 - acc: 1.000 - ETA: 6s - loss: 1.7118e-04 - acc: 1.000 - ETA: 6s - loss: 1.7219e-04 - acc: 1.000 - ETA: 6s - loss: 1.7195e-04 - acc: 1.000 - ETA: 6s - loss: 1.7004e-04 - acc: 1.000 - ETA: 6s - loss: 1.7276e-04 - acc: 1.000 - ETA: 5s - loss: 1.7343e-04 - acc: 1.000 - ETA: 5s - loss: 1.7187e-04 - acc: 1.000 - ETA: 6s - loss: 1.7150e-04 - acc: 1.000 - ETA: 5s - loss: 1.7014e-04 - acc: 1.000 - ETA: 5s - loss: 1.6969e-04 - acc: 1.000 - ETA: 5s - loss: 1.6912e-04 - acc: 1.000 - ETA: 5s - loss: 1.6812e-04 -

The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The keymap.all_axes rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The animation.avconv_path rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
The animation.avconv_args rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>